## Consumption of Pornographic Content by Age and Education

In [1]:
import pandas as pd

### Load Data

The 2004 comScore data are already grouped by machine_id and domain_name. The data has four columns: 
a. machine_id, 
b. domain_name, 
c. total duration of visits to the site, 
d. number of visits

Each row gives the total visits to and total duration spent on a domain.

We merge this with data from pydomains and the Trusted Source API. We then calculate the time spent on and total visits to pornographic domains based on a variety of measures. We also create two other columns that track proportion of visits, and proportion of time.

In [2]:
YEAR = 2004

In [3]:
# Read in data
idf = pd.read_csv('/opt/comscore/pydomains/app2/cs%04d_grp_machine_domain.csv.bz2' % YEAR)

In [4]:
# Load the pydomains data
pydom_df = pd.read_csv('/opt/comscore/pydomains/cs%04d_unique_domains_pydomains.csv.bz2' % YEAR, usecols=['domain_names', 'shalla_2017_cat', 'pred_shalla_2017_lab', 'pred_shalla_2017_prob_porn', 'pred_toulouse_2017_prob_adult'], encoding='latin1')
# rename column
pydom_df.rename(columns={'domain_names': 'domain_name'}, inplace=True)

In [5]:
# Load trusted data
tdf = pd.read_csv('/opt/comscore/pydomains/comScore_unique_2004.csv')
# rename column
tdf.rename(columns={'unique_url': 'domain_name', 'url_class': 'trusted_cat'}, inplace=True)

### Left join Trusted Source and PyDomains

In [6]:
# Left join with pydomain
pdf = idf.merge(pydom_df, how='left', on='domain_name')

# Left join with Trusted
pdf = pdf.merge(tdf, how='left', on='domain_name')

For domains that are already in the labeled datasets, we use the labels from there.

In [7]:
# -1 for Unknown, 0 - No, 1 - Yes
pdf['shalla_trusted_porn'] = 0
pdf.loc[pdf.shalla_2017_cat.isnull() & (pdf.trusted_cat.isnull() | (pdf.trusted_cat == 'UNKNOWN')), 'shalla_trusted_porn'] = -1
pdf.loc[pdf.shalla_2017_cat.notnull() & pdf.shalla_2017_cat.str.contains('porn', case=False), 'shalla_trusted_porn'] = 1
pdf.loc[pdf.trusted_cat.notnull() & pdf.trusted_cat.str.contains('porn', case=False), 'shalla_trusted_porn'] = 1

pdf[['shalla_2017_cat', 'trusted_cat', 'shalla_trusted_porn']].head()

,shalla_2017_cat,trusted_cat,shalla_trusted_porn
0,NaN,UNKNOWN,-1
1,porn|hobby/games-online,Games,1
2,NaN,Internet Services,0
3,NaN,Online Shopping,0
4,porn,Pornography,1


### Unique domain names

In [8]:
udf = pdf.drop_duplicates(subset='domain_name').copy()
# Total unique domains
len(udf)

1011145

### Total number of domains for which the label must be imputed

Curated lists generally only information about the kind of content hosted by a small fraction of domains. Commercial APIs generally are a lot better but still miss a sizable chunk. In Shallist, for instance, only about 22% of the domains in the data have category assigned to them (see below). For Trusted Source, the commensurate number is nearly 86%. In all, we know the category of about 86% of the domains.

In [9]:
# -1 for Unknown, 0 - No, 1 - Yes
udf['shalla_cat_porn'] = -1
udf.loc[udf.shalla_2017_cat.notnull() & (udf.shalla_2017_cat.str.contains('unknown', case=False) == False), 'shalla_cat_porn'] = 0
udf.loc[udf.shalla_2017_cat.notnull() & udf.shalla_2017_cat.str.contains('porn', case=False), 'shalla_cat_porn'] = 1
udf.groupby('shalla_cat_porn').agg({'domain_name': 'count'})/udf.shape[0]

,domain_name
shalla_cat_porn,
-1,0.780723
0,0.096061
1,0.123216


In [10]:
# -1 for Unknown, 0 - No, 1 - Yes
udf['trusted_cat_porn'] = -1
udf.loc[udf.trusted_cat.notnull() & (udf.trusted_cat.str.contains('unknown', case=False) == False), 'trusted_cat_porn'] = 0
udf.loc[udf.trusted_cat.notnull() & udf.trusted_cat.str.contains('porn', case=False), 'trusted_cat_porn'] = 1
udf.groupby('trusted_cat_porn').agg({'domain_name': 'count'})/udf.shape[0]

,domain_name
trusted_cat_porn,
-1,0.156131
0,0.691121
1,0.152748


In [11]:
udf.groupby('shalla_trusted_porn').agg({'domain_name': 'count'})/udf.shape[0]

,domain_name
shalla_trusted_porn,
-1,0.154937
0,0.674917
1,0.170146


### Impact of Different Cut-offs

Next, we use the labeled data (from Trusted and Shallalist) to pick different probability cut-offs to test how inferences changes. We choose three: one that minimizes FP+FN, one that gives us far fewer FP, and one that gives us far fewer FN. (We cast a wide net.)

To get the value that minimizes FP+FN for a particular category in a multi-class prediction problem, we [run an optimization algorithm](https://github.com/soodoku/optimal_softmax_cutoffs).

In [12]:
# Filter out unknown (-1)
odf = udf[udf.shalla_trusted_porn != -1][['shalla_trusted_porn', 'pred_shalla_2017_prob_porn', 'pred_toulouse_2017_prob_adult']].copy()
odf.head()

,shalla_trusted_porn,pred_shalla_2017_prob_porn,pred_toulouse_2017_prob_adult
1,1,0.240971,0.341799
2,0,0.854591,0.996538
3,0,0.109746,0.115253
4,1,0.979598,0.985338
5,0,0.194238,0.406939


In [29]:
import optimal_cut_offs

optimal_cut_offs.get_probability(odf.shalla_trusted_porn, odf.pred_shalla_2017_prob_porn, 'accuracy', verbose=True)

Probability: 0.1000 Accuracy: 0.45
Probability: 0.1000 Accuracy: 0.45
Probability: 0.2000 Accuracy: 0.62
Probability: 0.3000 Accuracy: 0.74
Probability: 0.4000 Accuracy: 0.81
Probability: 0.5000 Accuracy: 0.85
Probability: 0.6000 Accuracy: 0.88
Probability: 0.7000 Accuracy: 0.89
Probability: 0.8000 Accuracy: 0.90
Probability: 0.8000 Accuracy: 0.90
Probability: 0.8400 Accuracy: 0.91
Probability: 0.8800 Accuracy: 0.91
Probability: 0.9200 Accuracy: 0.91
Probability: 1.0000 Accuracy: 0.80
Probability: 0.8800 Accuracy: 0.91
Probability: 0.9600 Accuracy: 0.91
Probability: 0.9000 Accuracy: 0.91
Probability: 0.8800 Accuracy: 0.91
Probability: 0.9100 Accuracy: 0.91
Probability: 0.9200 Accuracy: 0.91
Probability: 0.9050 Accuracy: 0.91
Probability: 0.9150 Accuracy: 0.91
Probability: 0.9075 Accuracy: 0.91
Probability: 0.9050 Accuracy: 0.91
Probability: 0.9088 Accuracy: 0.91
Probability: 0.9100 Accuracy: 0.91
Probability: 0.9081 Accuracy: 0.91
Probability: 0.9075 Accuracy: 0.91
Probability: 0.9084 

0.9081250000000004

In [30]:
optimal_cut_offs.get_probability(odf.shalla_trusted_porn, odf.pred_shalla_2017_prob_porn, 'f1', verbose=True)

Probability: 0.1000 F1 score: 0.41
Probability: 0.1000 F1 score: 0.41
Probability: 0.2000 F1 score: 0.49
Probability: 0.3000 F1 score: 0.58
Probability: 0.4000 F1 score: 0.64
Probability: 0.5000 F1 score: 0.69
Probability: 0.6000 F1 score: 0.71
Probability: 0.7000 F1 score: 0.73
Probability: 0.8000 F1 score: 0.74
Probability: 0.8000 F1 score: 0.74
Probability: 0.8400 F1 score: 0.74
Probability: 0.8800 F1 score: 0.74
Probability: 0.8200 F1 score: 0.74
Probability: 0.8000 F1 score: 0.74
Probability: 0.8300 F1 score: 0.74
Probability: 0.8100 F1 score: 0.74
Probability: 0.8250 F1 score: 0.74
Probability: 0.8250 F1 score: 0.74
Probability: 0.8150 F1 score: 0.74
Probability: 0.8225 F1 score: 0.74
Probability: 0.8250 F1 score: 0.74
Probability: 0.8213 F1 score: 0.75
Probability: 0.8200 F1 score: 0.74
Probability: 0.8219 F1 score: 0.74
Probability: 0.8219 F1 score: 0.74
Probability: 0.8206 F1 score: 0.75
Probability: 0.8200 F1 score: 0.74
Probability: 0.8200 F1 score: 0.74
Probability: 0.8209 

0.820546875